<a href="https://colab.research.google.com/github/canfielder/DSBA-6190_Proj4_Serverless-Pipeline/blob/master/notebooks/Lambda_Output_Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

## Install

In [0]:
!pip -q install boto3

## Packages

In [0]:
# General
import os
import pandas as pd
import pprint

# AWS Connection
from google.colab import drive
import boto3


# Set Up AWS Connection

## Mount Google Drive

In [7]:
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "/content/gdrive/My Drive/aws/credentials/"

aws_dir = os.listdir(path)

aws_credentials = aws_dir.pop(0)

Establish the source location of the AWS credential file.

In [9]:
aws_credentials_src = os.path.join(path, aws_credentials)
aws_credentials_src = aws_credentials_src.replace(" ", "\ ")
print(aws_credentials_src)

/content/gdrive/My\ Drive/aws/credentials/credentials


Establish the destination of where to copy the AWS credential file.

In [10]:
aws_credentials_dst = "~/.aws/credentials"
print(aws_credentials_dst)    

~/.aws/credentials


Copy the credentials from the my mounted Google Drive to the local folder.

In [0]:
#!/usr/bin/env python3
mkdir -p ~/.aws &&\
  cp -r {aws_credentials_src} {aws_credentials_dst} 

Verify the credentials were correctly copied.

In [12]:
#!/usr/bin/env python3
ls -R {aws_credentials_dst}

/root/.aws/credentials


# Establish Boto3 Session

By establishing a Boto3 session with Region, all downstream uses of Boto3 will import these associated values. We do not have to define the region multiple times.

In [0]:
profile = 'dsba_6190_proj_4'
region = 'us-east-1'

session = boto3.Session(profile_name=profile, region_name=region)

# Processing

## Generate List of All Files

In [0]:
s3 = session.client(service_name="s3")

The following code chunk comes from this blog post, accessed on 3/16/2020:

[https://alexwlchan.net/2017/07/listing-s3-keys/](https://alexwlchan.net/2017/07/listing-s3-keys/)


The code generates a list of all files in a bucket. It is not recursive. An issue with S3 buckets is 

In [0]:
def get_all_s3_keys(bucket):
    """Get a list of all keys in an S3 bucket."""
    keys = []

    kwargs = {'Bucket': bucket}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            keys.append(obj['Key'])

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return keys

In [34]:
get_all_s3_keys(bucket)

['entity_barkley_marathons.csv',
 'entity_hardrock_hundred_mile_endurance_run.csv',
 'entity_leadville_trail_100.csv',
 'entity_ultra-trail_du_mont-blanc.csv',
 'entity_vermont_100_mile_endurance_run.csv',
 'entity_western_states_endurance_run.csv']